# Grassmann Kernels

For more information see [Expanding the Family of Grassmannian Kernels: An Embedding Perspective](https://arxiv.org/abs/1407.1123). 

Suppose we are given the following list of kernels on the Grassmannian:
1. Polynomial: $K_{p, bc}(X, Y) = (\beta + |\det(X^T Y)|)^{\alpha}, \beta >0$
2. RBF: $K_{r, bc}(X, Y) = \exp(\beta|\det(X^T Y)|), \beta >0$
3. Laplace: $K_{l, bc}(X, Y) = \exp(-\beta \sqrt{1-|\det(X^T Y)|}), \beta >0$
4. Binomial: $K_{bi, bc}(X, Y) = (\beta - |\det(X^T Y)|)^{\alpha}, \beta >1$
5. Lorgarithm: $K_{log, bc}(X, Y) = -\log(2 - |\det(X^T Y)|)$

Where we use the inner product $\langle X, Y \rangle_P =  |P(X)^TP(Y)| = |\det(X^T Y)|$ as the inner product on the Plücker embeddings $P(X)$ and $P(Y)$. This inner product induces the distance metric $d_{bc}(X, Y) = ||P(X) - P(Y)||^2 = 2 - 2|\det(X^T Y)|$, which is the same as the geodesic distance $d_g(X, Y)$ up to scale $\sqrt{2}$. Please explain how this might be used in our context. 

The provided list of kernels provides different ways of calculating the similarity between points on the Grassmannian, using the Plücker embedding. These kernels can be used in machine learning algorithms to compute the similarity between subspaces represented by the matrices $X$ and $Y$.

The choice of kernel depends on the nature of your data and the problem you are trying to solve. Here is a high-level description of how each kernel could be useful:

1. **Polynomial Kernel**: This kernel measures the similarity between $X$ and $Y$ as a power of their inner product plus a constant $\beta$. This can be useful when you want to capture more complex relationships between your data points that might not be linearly separable in the original space.

2. **RBF (Radial Basis Function) Kernel**: This is an exponential kernel, which can capture a wide range of relationships between data points. It is one of the most commonly used kernels in machine learning and can handle non-linear relationships well.

3. **Laplace Kernel**: The Laplace kernel is similar to the RBF kernel but with the difference that it is less sensitive to changes in the determinant of $X^T Y$. This can make it more robust to variations in the data.

4. **Binomial Kernel**: This kernel measures similarity as a power of the difference between a constant $\beta$ and the inner product. Like the polynomial kernel, it can capture complex relationships between data points.

5. **Logarithm Kernel**: The logarithm kernel is unique in the sense that it directly uses the logarithm of the inner product. This can be useful if the relationships between your data are best captured by a logarithmic function.

The distance metric $d_{bc}(X, Y)$ is derived from the inner product and essentially measures the "distance" between points in the Grassmannian in terms of the determinant of their inner product. This distance is equivalent to the geodesic distance, up to a scaling factor of $\sqrt{2}$. The geodesic distance is the shortest path between two points in a curved space, like the Grassmannian.

In the context of machine learning, these kernels and the distance metric can be used in many algorithms that involve measuring similarity or distance between data points. For example, they can be used in support vector machines (SVMs), kernelized regression, or clustering algorithms operating on data points in the Grassmannian. The choice of kernel can significantly influence the performance of these algorithms, so it would be important to choose a kernel that is appropriate for your specific task and data.

In [32]:
import numpy as np

def gram_schmidt(X):
    Q, R = np.linalg.qr(X)
    return Q

def polynomial_kernel(X, Y, alpha, beta):
    Q_X = gram_schmidt(X)
    Q_Y = gram_schmidt(Y)
    inner_product_det = np.abs(np.linalg.det(Q_X.T @ Q_Y))
    return (beta + inner_product_det) ** alpha

def rbf_kernel(X, Y, beta):
    Q_X = gram_schmidt(X)
    Q_Y = gram_schmidt(Y)
    inner_product_det = np.abs(np.linalg.det(Q_X.T @ Q_Y))
    return np.exp(-beta * (1 - inner_product_det))

def laplace_kernel(X, Y, beta):
    Q_X = gram_schmidt(X)
    Q_Y = gram_schmidt(Y)
    inner_product_det = np.abs(np.linalg.det(Q_X.T @ Q_Y))
    return np.exp(-beta * np.sqrt(1 - inner_product_det))

def binomial_kernel(X, Y, alpha, beta):
    Q_X = gram_schmidt(X)
    Q_Y = gram_schmidt(Y)
    inner_product_det = np.abs(np.linalg.det(Q_X.T @ Q_Y))
    return (alpha - inner_product_det) ** beta

def logarithm_kernel(X, Y):
    Q_X = gram_schmidt(X)
    Q_Y = gram_schmidt(Y)
    inner_product_det = np.abs(np.linalg.det(Q_X.T @ Q_Y))
    return -np.log(2 - inner_product_det)

def bc_distance(X, Y):
    Q_X = gram_schmidt(X)
    Q_Y = gram_schmidt(Y)
    inner_product_det = np.abs(np.linalg.det(Q_X.T @ Q_Y))
    return np.arccos(np.sqrt(inner_product_det))



In [33]:
X = np.random.rand(3,2)
Y = np.random.rand(3,2)
alpha = 2
beta = 3
bc_distance(X, Y)


0.8196546783111077

In [34]:
polynomial_kernel(X, Y, alpha, beta)

12.011563626881381

In [35]:
rbf_kernel(X, Y, beta)

0.20135431471900322

In [36]:
laplace_kernel(X, Y, beta)

0.11161156547948795

In [37]:
binomial_kernel(X, Y, alpha, beta)

3.611363254832913

In [38]:
logarithm_kernel(X, Y)

-0.4280284446590432

Before starting the discussion, let's introduce some notations and assumptions.

- $\Delta W_i^{(n)}$: The $n^{th}$ layer's update weight matrix in the $i^{th}$ Low Rank Adaptation (LoRA) model. Assume there are $K$ models, and each model has $N$ layers. Thus, $i \in \{1,2,...,K\}$ and $n \in \{1,2,...,N\}$.
- $\{Gr(p, V)\}$: A Grassmann manifold, which is a space of $p$-dimensional linear subspaces of $V$.

Let's denote the kernel function as $k(\cdot, \cdot)$, which can be any of the five kernels mentioned above (Polynomial, RBF, Laplace, Binomial, or Logarithm). The kernel function calculates the similarity between two points on the Grassmann manifold, represented by their respective U matrices obtained from the SVD of the update weight matrices.

The first step in clustering involves calculating the pairwise similarity between all points. This gives us a similarity matrix $K$ where the element $K_{ij}$ is the similarity between points $i$ and $j$, calculated using the kernel function on the Grassmann manifold.

$$ K_{ij} = k(U_{\Delta W_i^{(n)}}, U_{\Delta W_j^{(m)}}) $$

After calculating the similarity matrix, we can apply a clustering algorithm, such as k-means or spectral clustering, directly on the similarity matrix. 

In the case of k-means, we aim to minimize the within-cluster sum of squares (WCSS), which is the sum of the squared distances between each point and the centroid of its assigned cluster. In our context, the distance between two points is defined in terms of the Binet-Cauchy distance, as follows:

$$ d_{bc}(\Delta W_i^{(n)}, \Delta W_j^{(m)}) = 2 - 2 \cdot \left| \det(U_{\Delta W_i^{(n)}}^T U_{\Delta W_j^{(m)}}) \right| $$

The k-means clustering algorithm involves the following steps:

1. Initialize $k$ centroids randomly.
2. Assign each point to the cluster that has the closest centroid.
3. Update the centroids by calculating the geometric mean of all points in the cluster.
4. Repeat steps 2 and 3 until convergence.

On the other hand, spectral clustering uses the eigenvectors of the Laplacian matrix, which is defined based on the similarity matrix, to perform dimensionality reduction before applying a standard clustering algorithm like k-means.

While these steps outline the general process of clustering, please note that clustering on Grassmann manifolds involves additional complexities due to the non-Euclidean nature of the space. These complexities may require the use of specialized algorithms or adaptations of existing algorithms to effectively work with the Grassmann manifold geometry.

Moreover, the process of finding the geometric mean (or centroid) in a Grassmann manifold is not straightforward and involves the minimization of a cost function defined over the manifold, such as the sum of squared geodesic distances to all points in the cluster. This process may involve iterative methods or the use of optimization techniques that take into account the manifold's structure.

Finally, remember to validate the assumptions and constraints of the specific application, and make sure to choose the kernel and clustering method that best suit the problem at hand.

DBSCAN (Density-Based Spatial Clustering of Applications with Noise) is a density-based clustering algorithm, which groups together points in dense regions and separates points in sparse regions.

The DBSCAN algorithm requires two parameters:

1. `eps`: the maximum distance between two samples for them to be considered as in the same neighborhood.
2. `min_samples`: the number of samples in a neighborhood for a point to be considered as a core point.

In the context of Grassmann manifolds, we'll use the Binet-Cauchy distance $d_{bc}$ as defined previously:

$$ d_{bc}(X, Y) = 2 - 2 \cdot \left| \det(X^T Y) \right| $$

Here is how the DBSCAN algorithm can be applied in our context:

1. For each point $p$ in our dataset, we compute the neighborhood $N_{eps}(p)$, which is the set of points whose distance to $p$ is less than `eps`:

$$ 
N_{\epsilon}(p) = \{q : d_{bc}(p, q) \leq \epsilon \} 
$$

2. If a point $p$ has at least `min_samples` in its `eps`-neighborhood (including $p$ itself), we consider $p$ as a core point. Formally, $p$ is a core point if:

$$ 
\left| N_{\epsilon}(p)\right| \geq \text{min\_samples} 
$$

3. We create a cluster for each core point and its reachable points. A point $q$ is directly reachable from a point $p$ if $q$ is within `eps` distance from $p$ and $p$ is a core point. A point $q$ is reachable from $p$ if there is a sequence of points $p_1, ..., p_n$ with $p_1=p$ and $p_n=q$ such that each point $p_{i+1}$ is directly reachable from $p_i$.

4. Assign each point to the cluster of the nearest core point. If a point is not reachable from any core point, it's considered as noise.

In practice, DBSCAN can be sensitive to the choice of `eps` and `min_samples` parameters, and different choices can lead to very different clustering results. It's often recommended to normalize the data before applying DBSCAN, but in the context of Grassmann manifolds, the points are already on a unit sphere due to the properties of the SVD.

Kernel methods can be used for clustering in a variety of ways. The basic idea is to use the kernel function to map the input data into a higher-dimensional space, where clusters might be more easily discernible. 

One popular method is Kernel K-means, which is a variant of the K-means clustering algorithm. In the standard K-means algorithm, the goal is to partition the data into K clusters such that the within-cluster sum of squares is minimized:

$$ \min_{S} \sum_{i=1}^{k} \sum_{\mathbf{x} \in S_i} \|\mathbf{x} - \mathbf{\mu}_i\|^2 $$

where $\mu_i$ is the mean of points in $S_i$.

In the Kernel K-means version, we replace the Euclidean distance with a distance measure in the feature space induced by a kernel function $K$. This is equivalent to performing K-means in this higher-dimensional space, and the objective function becomes:

$$ \min_{S} \sum_{i=1}^{k} \sum_{\mathbf{x}, \mathbf{z} \in S_i} K(\mathbf{x}, \mathbf{x}) + K(\mathbf{z}, \mathbf{z}) - 2K(\mathbf{x}, \mathbf{z}) $$

The Spectral Clustering method also uses kernel matrices. It uses the eigenvectors of the Laplacian of the kernel matrix to perform dimensionality reduction, then applies a clustering algorithm (like K-means) in the reduced space.

The Laplacian $L$ of a kernel matrix $K$ is computed as $L = D - K$, where $D$ is a diagonal matrix where $D_{ii}$ is the sum of the $i$-th row of $K$. After computing $L$, the algorithm finds the $k$ smallest eigenvectors, and forms a new matrix where the $i$-th row is the $i$-th eigenvector. Finally, K-means is performed on this new matrix to get the clusters.

In [39]:
import numpy as np
from scipy.linalg import subspace_angles

def gram_schmidt(X):
    Q, R = np.linalg.qr(X)
    return Q

def polynomial_kernel(X, Y, alpha, beta):
    Q_X = gram_schmidt(X)
    Q_Y = gram_schmidt(Y)
    inner_product_det = np.abs(np.linalg.det(Q_X.T @ Q_Y))
    return (beta + inner_product_det) ** alpha

# Number of subspaces (i.e., data points)
num_subspaces = 5

# Dimension of ambient space
ambient_dim = 5

# Dimension of each subspace
subspace_dim = 3

# Generate a list of random subspaces
X = [np.random.randn(ambient_dim, subspace_dim) for _ in range(num_subspaces)]

alpha = 2
beta = 3

# Initialize an empty matrix for the kernel matrix
K = np.zeros((num_subspaces, num_subspaces))

for i in range(num_subspaces):
    for j in range(num_subspaces):
        K[i, j] = polynomial_kernel(X[i], X[j], alpha, beta)

print(K)



[[16.          9.2664217  10.1434243  10.65910497 11.30017748]
 [ 9.2664217  16.         12.04407451 10.26218048 10.0785779 ]
 [10.1434243  12.04407451 16.          9.38791207 11.01756676]
 [10.65910497 10.26218048  9.38791207 16.          9.75727678]
 [11.30017748 10.0785779  11.01756676  9.75727678 16.        ]]


In [42]:
from safetensors import safe_open
from scipy.linalg import svd
import numpy as np

# Load the LoRA tensors from .safetensors files
with safe_open("fashigirl-v5.5-lora-naivae-64dim.safetensors", framework="pt", device="cpu") as f:
    lora1_tensors = {}
    for k in f.keys():
        lora1_tensors[k] = f.get_tensor(k)

with safe_open("lora2.safetensors", framework="pt", device="cpu") as f:
    lora2_tensors = {}
    for k in f.keys():
        lora2_tensors[k] = f.get_tensor(k)

In [43]:
import torch

# Gather all keys and sort them
all_keys = sorted(list(lora1_tensors.keys()))

# Filter keys for lora_down and lora_up pairs
lora_down_keys = [key for key in all_keys if 'lora_down' in key]
lora_up_keys = [key for key in all_keys if 'lora_up' in key]

# Ensure we have matching pairs of keys
assert len(lora_down_keys) == len(lora_up_keys), "Mismatch in number of 'lora_down' and 'lora_up' keys"

# Define the subspace similarity measure
def phi(U1, U2, i, j):
    U1_i = U1[:, :i]  # First i columns of U1
    U2_j = U2[:, :j]  # First j columns of U2
    
    product = torch.matmul(U1_i.t(), U2_j)  # Matrix multiplication
    norm = torch.norm(product)  # Frobenius norm
    
    return norm ** 2 / min(i, j)

# Iterate over all layers
for layer in range(len(lora_down_keys)):
    try:
        # Extract the corresponding A and B matrices
        A1 = lora1_tensors[lora_down_keys[layer]].float()
        B1 = lora1_tensors[lora_up_keys[layer]].float()

        A2 = lora2_tensors[lora_down_keys[layer]].float()
        B2 = lora2_tensors[lora_up_keys[layer]].float()

        # Print the shapes of A1 and B1 matrices for troubleshooting
        print(f"A1 shape: {A1.shape}")
        print(f"B1 shape: {B1.shape}")

        print(f"A2 shape: {A2.shape}")
        print(f"B2 shape: {B2.shape}")

        # Compute the update matrices
        Delta_W1 = torch.matmul(B1, A1)
        print(f"ΔW1 shape: {Delta_W1.shape}")
        Delta_W2 = torch.matmul(B2, A2)
        print(f"ΔW2 shape: {Delta_W2.shape}")


        # Compute the SVD of the update matrices
        U1, _, _ = torch.svd(Delta_W1)
        U2, _, _ = torch.svd(Delta_W2)

        # Calculate the subspace similarity measure
        i = U1.size(1)  # Number of columns in U1
        j = U2.size(1)  # Number of columns in U2
        alpha = 2
        beta = 3
        result = polynomial_kernel(U1, U2, alpha, beta)  # Replace i and j with the desired values

        print(f"The polynomial kernel similarity measure for layer {layer} is: {result}")

    except RuntimeError as e:
        # Print the layer number and the error message
        print(f"Error occurred at layer {layer}: {e}")

A1 shape: torch.Size([64, 768])
B1 shape: torch.Size([3072, 64])
A2 shape: torch.Size([64, 768])
B2 shape: torch.Size([3072, 64])
ΔW1 shape: torch.Size([3072, 768])
ΔW2 shape: torch.Size([3072, 768])
The polynomial kernel similarity measure for layer 0 is: 9.0
A1 shape: torch.Size([64, 3072])
B1 shape: torch.Size([768, 64])
A2 shape: torch.Size([64, 3072])
B2 shape: torch.Size([768, 64])
ΔW1 shape: torch.Size([768, 3072])
ΔW2 shape: torch.Size([768, 3072])
The polynomial kernel similarity measure for layer 1 is: 15.999998092651424
A1 shape: torch.Size([64, 768])
B1 shape: torch.Size([768, 64])
A2 shape: torch.Size([64, 768])
B2 shape: torch.Size([768, 64])
ΔW1 shape: torch.Size([768, 768])
ΔW2 shape: torch.Size([768, 768])
The polynomial kernel similarity measure for layer 2 is: 16.00000190734869
A1 shape: torch.Size([64, 768])
B1 shape: torch.Size([768, 64])
A2 shape: torch.Size([64, 768])
B2 shape: torch.Size([768, 64])
ΔW1 shape: torch.Size([768, 768])
ΔW2 shape: torch.Size([768, 76